# View postISRCCD from DM PostISRCCD 

- work with Weakly_2023_35

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2023/09/20
- last update : 2023/09/20


In [ ]:
! eups list -s | grep LOCAL

In [ ]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib.dates as mdates

import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch


In [ ]:
# LSST Display
import lsst.afw.display as afwDisplay
afw_display = afwDisplay.getDisplay(backend='firefly')

In [ ]:
import lsst.daf.butler as dafButler
from lsst.daf.butler import CollectionType

In [ ]:
#repo = "/sdf/group/rubin/repo/main" # repo/main
repo="/sdf/group/rubin/repo/oga/"  # /repo/embargo
butler = dafButler.Butler(repo)
registry = butler.registry

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Registry

In [ ]:
butler.registry.dimensions.getDatabaseElements()

In [ ]:
print(butler.registry.dimensions["exposure"].RecordClass.fields)

In [ ]:
print(butler.registry.dimensions["visit"].RecordClass.fields)

In [ ]:
print(butler.registry.dimensions["physical_filter"].RecordClass.fields)

In [ ]:
print(butler.registry.dimensions["tract"].RecordClass.fields)

# Configuration

In [ ]:
FLAG_PLOT=True

In [ ]:
#DATE = 20230912
#FILTER="cyl_lens~holo4_001"
#FILTER="cyl_lens~holo4_003"

#my_collection = "u/dagoret/spectro/collimatorholo4003noflat/runs_20230912"
#FILTER="collimator~holo4_003"
#my_collection = "u/dagoret/spectro/emptyholo4003noflat/runs_20230912"
#FILTER="empty~holo4_003"
#my_collection = "u/dagoret/spectro/emptyholo4001noflat/runs_20230912"
#FILTER="empty~holo4_001"


#DATE = 20230913
#my_collection = "u/dagoret/spectro/collimatorholo4003noflat/runs_20230913"
#FILTER="collimator~holo4_003"
#my_collection = "u/dagoret/spectro/emptyholo4003noflat/runs_20230913"
#FILTER="empty~holo4_003"
#my_collection = "u/dagoret/spectro/emptyholo4001noflat/runs_20230913"
#FILTER="empt~holo4_001"



DATE = 20230914
my_collection = "u/dagoret/spectro/collimatorholo4003noflat/runs_20230914"
FILTER="collimator~holo4_003"
#my_collection = "u/dagoret/spectro/emptyholo4003noflat/runs_20230914"
#FILTER="empty~holo4_003"
#my_collection = "u/dagoret/spectro/emptyholo4001noflat/runs_20230914"
#FILTER="empty~holo4_001"


# read list of exposures

From the butler and the given user collection

In [ ]:
datasetRefs = registry.queryDatasets(datasetType='postISRCCD', collections=my_collection, where= "instrument='LATISS'")

In [ ]:
all_dataId = []
all_postisrccd  = []
all_exposures = []
for i, ref in enumerate(datasetRefs):
   
    print(f"========({i})================datasetType = postISRCCD ============================================")
    print("fullId..................:",ref.dataId.full)
    print("exposure................:",ref.dataId["exposure"])
    print("band....................:",ref.dataId["band"])
    print("physical filter.........:",ref.dataId["physical_filter"])
    print("run.....................:",ref.run)
    the_exposure = ref.dataId["exposure"]
    the_day_obs = ref.dataId["exposure"]//100_000
    the_seq_num = ref.dataId["exposure"]- the_day_obs*100_000    
    the_dataId = {'day_obs': the_day_obs,'seq_num':the_seq_num,'detector':0}
    print(the_dataId)
    #spec       = butler.get('spectraction',the_dataId)
    postisrccd = butler.get('postISRCCD', exposure=the_exposure, detector=0, collections=my_collection, instrument='LATISS')
    all_dataId.append(the_dataId) 
    all_exposures.append(the_exposure)
    all_postisrccd.append(postisrccd)
all_exposures = np.array(all_exposures)

In [ ]:
N = len(all_postisrccd)

In [ ]:
all_exposures_sortedindexes = np.argsort(all_exposures)

In [ ]:
all_exposures[all_exposures_sortedindexes]

In [ ]:
#for idx in range(N):
#    idx_sorted = all_exposures_sortedindexes[idx]
#    dm_postisrccd = all_postisrccd[idx_sorted]
#    dm_postisrccd_md = dict(dm_postisrccd.getMetadata())
#    print(f"==================={all_exposures[idx_sorted]}=======================")
#    print('LSST CALIB DATE BIAS :',dm_postisrccd_md['LSST CALIB DATE BIAS'])
#    print('LSST CALIB UUID BIAS :',dm_postisrccd_md['LSST CALIB UUID BIAS']) 
#    print('LSST CALIB RUN BIAS  :',dm_postisrccd_md['LSST CALIB RUN BIAS'])
#    print("-------")
#    print('LSST CALIB DATE DEFECTS :',dm_postisrccd_md['LSST CALIB DATE DEFECTS'])
#    print('LSST CALIB UUID DEFECTS :',dm_postisrccd_md['LSST CALIB UUID DEFECTS']) 
#    print('LSST CALIB RUN DEFECTS  :',dm_postisrccd_md['LSST CALIB RUN DEFECTS'])

## Select flags options

In [ ]:
FLAG_ROTATE_IMG = True
FLAG_TRANSFORM = True

## Transformations
astropy scale transformations

In [ ]:
transform = AsinhStretch() + PercentileInterval(99.)
#transform = PercentileInterval(98.)

In [ ]:
idx=10
idx_sorted = all_exposures_sortedindexes[idx]
dm_postisrccd = all_postisrccd[idx_sorted]
dm_postisrccd_md = dict(dm_postisrccd.getMetadata())
exposure_selected =all_exposures[idx_sorted]

print(exposure_selected)
raw_img= butler.get('raw', dataId={'exposure': exposure_selected, 'instrument': 'LATISS', 'detector': 0}, collections = ['LATISS/calib','LATISS/raw/all',] )
meta = raw_img.getMetadata()
md = meta.toDict()
      
the_object = md['OBJECT']
the_am= md['AMSTART']
the_filter=md['FILTER']

In [ ]:
afw_display.scale('linear', 'zscale',None)
the_title = f"{idx} :: dm postisrccd exposure : {exposure_selected}, target={the_object}, airmass={the_am:.2f}, filter={the_filter}"
#le titre ne marche pas avec firefly
#afw_display.mtv(dm_postisrccd.image,title=the_title)
afw_display.mtv(dm_postisrccd.image)

In [ ]:
print("finished")